In [1]:
import pandas as pd
import numpy as np
import requests
import urllib
import urllib3
import certifi
import json
import sqlite3
import os
import re

from bs4 import BeautifulSoup
import unidecode

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# prep 'http'
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())


In [2]:
spot_id = '4263d6a900e94f1599974e3c90c28aa5'
spot_sec = '26333e019ca149a4b8f0f1633168dddd'
rc_url = 'http://api.spotify.com/v1/users/spotify/playlists/37i9dQZF1DX0XUsuxWHRQd'

In [3]:
def pnoun_test(t):
    m_count = 0
    f_count = 0
    m = ['he', 'him', 'his', 'himself']
    f = ['she', 'her', 'hers', 'herself']
    for w in t.split():
        if w in m: 
            m_count += 1
        if w in f:
            f_count += 1
    return([m_count, f_count])

In [4]:
# returns a bio or NaN if none exists
def searchy(name, group=False):
    # execute search
    url = f'https://www.allmusic.com/search/artists/{"%20".join(name.split())}'

    # make request
    r = http.request('GET', url)

    # if response is good, scrape biography
    if r.status == 200:
        soup = BeautifulSoup(r.data, 'lxml')
        link = soup.find('div', {'class':'name'}).find('a')['href']
        
        r2 = http.request('GET', (link + '/biography/'))
        if r2.status == 200:
            soup2 = BeautifulSoup(r2.data, 'lxml')
            
            # if group, pull group members instead of bio
            if group:
                try:
                    members = []
                    for m in soup2.find('div', {'class':'group-members'}).find_all('a'):
                        members.append(m.text.strip())
                    return members
                
                # return NaN if no members
                except AttributeError:
                    return None
            
            # else pull bio
            else:
                try:
                    return soup2.find('div', {'itemprop':'reviewBody'}).text

                # return NaN if no biography entry
                except AttributeError:
                    return None

In [5]:
import json

client_credentials_manager = SpotifyClientCredentials(spot_id, spot_sec)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

uri = 'spotify:user:spotify:playlist:37i9dQZF1DX0XUsuxWHRQd'
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

results = sp.user_playlist(username, playlist_id)
rc_j = json.dumps(results, indent=4)
rc_jd = (json.loads(rc_j))

In [31]:
# non-SQL version
ats = {}
trax = []
for n in range(50):
    a = rc_jd['tracks']['items'][n]['track']['artists'][0]['name']
    trax.append(rc_jd['tracks']['items'][n]['track']['name'])
    
    if a not in ats:
        ats[a] = {'count':1}
    else:
        ats[a]['count'] += 1
        
for t in trax:
    if 'feat.' in t:
        feat = re.findall('(?<=feat. ).*[^)]', t)
    elif 'ft.' in t:
        feat = re.findall('(?<=ft. ).*[^)]', t)
    else:
        continue
    for n in re.split('[,&]', feat[0]):
        name = n.strip()
        if name not in ats:
            ats[name] = {'count':1}
        else:
            ats[name]['count'] += 1
    
group_members = False
for name in ats.keys():
#     print(name)
    
    # pull bio
    bio = searchy(name)
    
    # if no bio, tag as None
    if bio == None:
        ats[name]['gender'] = None
    
    # else run pronoun test
    else:
        g_zult = pnoun_test(bio)
#         print(g_zult)
    
        # if tied, pull members to re-run
        if g_zult[0] == g_zult[1]:
            members = searchy(name, group=True)
            if not group_members:
                group_members = []
            for m in members:
                group_members.append(unidecode.unidecode(m))
            ats[name]['gender'] = None
            continue
         
        # else, score and tag
        else:
            if g_zult[0] > g_zult[1]:
                ats[name]['gender'] = 'M'
            elif g_zult[1] > g_zult[0]:
                ats[name]['gender'] = 'F'
                
# if group_members, re-run
if group_members:
    for name in group_members:
        
        # if already in list, just advance counter
        if name in ats:
            ats[name]['count'] += 1
            
        # else, run the whole damn gendering script
        else:
            # pull bio
            bio = searchy(name)

            # if no bio, tag as None
            if bio == None:
                ats[name]['gender'] = None
                
            # else run pronoun test
            # i guess ill have to figure out if groups end up here at some point but wtv
            else:
                g_zult = pnoun_test(bio)
                if g_zult[0] > g_zult[1]:
                    ats[name] = {'count':1, 'gender':'M'}
                elif g_zult[1] > g_zult[0]:
                    ats[name] = {'count':1, 'gender':'F'}
    
# calculate stats
total = 0
M_tot = 0
F_tot = 0
for k, v in ats.items():
    c = v['count']
    total += c
    if v['gender'] == 'M':
        M_tot += c
    elif v['gender'] == 'F':
        F_tot += c
    
print(f'male credits: {M_tot}')
print(f'female credits: {F_tot}')
print(f'total credits: {total}')

male credits: 68
female credits: 5
total credits: 76


In [33]:
trax

['W O R K I N M E',
 'STARGAZING',
 'Album of the Year - Freestyle',
 'Barbie Dreams',
 'Kept Back (feat. Lil Pump)',
 'Taking A Walk',
 'L A M B T A L K',
 'FEFE (feat. Nicki Minaj & Murda Beatz)',
 'Nonstop',
 'Mo Bamba',
 'SICKO MODE',
 'SWISH',
 'Chun Swae (feat. Swae Lee)',
 'In My Feelings',
 'CAROUSEL',
 'Moonlight',
 'Pull Up (feat. NAV)',
 'Shake It Up',
 'Praise The Lord (Da Shine)',
 'Leave Me Alone',
 'Hopeless Romantic (feat. Swae Lee)',
 'Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj',
 'Taste (feat. Offset)',
 'Wasted (feat. Lil Uzi Vert)',
 'Leave Me',
 'Fr Fr (feat. Lil Skies)',
 'Yes Indeed',
 'Nephew (feat. Lil Pump)',
 'BULLETPROOF',
 'I Might Need Security',
 'Mob Ties',
 'Ned Flanders (feat. A$AP Ferg)',
 'Talk To Me (with Rich The Kid)',
 'Champion (feat. Travis Scott)',
 'Lost It (feat. Quavo & Offset)',
 'SAD!',
 'Lucid Dreams',
 'Life Goes On',
 'I Know You (feat. Yung Pinch)',
 'GHOST',
 'Millidelphia (feat. Swizz Beatz)',
 'WIN',
 'APESHIT',
 '2 Seater (fea

In [79]:
conn = sqlite3.connect('rcg_test.db')
c = conn.cursor()
a = "Migos"
b = "The Carters"
d = "Lil Pump"

In [66]:
qa.fetchone()[0]

'Beyoncé, Jay-Z'

In [56]:
c.fetchone() == None

True

In [84]:
qc = c.execute('SELECT gender FROM genders WHERE name=?', (d,))

In [85]:
qc.fetchone()[0]

'M'

In [90]:
qc2 = c.execute('SELECT members FROM groups WHERE name=?', (d,))

In [8]:
sqlite3.connections.close_all()

AttributeError: module 'sqlite3' has no attribute 'connections'

In [9]:
conn = sqlite3.connect('rcg_test.db')
c = conn.cursor()
c.execute('INSERT INTO genders (name, gender) VALUES ("Nicki Minaj", "F")')
conn.close()

OperationalError: database is locked

In [ ]:
# SQL version CHECK DATA FIRST
ats = {}
trax = []

conn = sqlite3.connect('rcg_test.db')
c = conn.cursor()

# add data for new artists
for n in range(50):
    a = rc_jd['tracks']['items'][n]['track']['artists'][0]['name']
    
    # query for new entry and for group status
    q = (a,)
    gen_test = c.execute('SELECT gender FROM genders WHERE name=?', q)
    gp_test = c.execute('SELECT members FROM groups WHERE name=?', q)
    
    gender = gen_test.fetchone()
    members = gp_test.fetchone()
    
    ### come back and redo this with "EXISTS"
    # if name isnt in gender table ...
    if gender == None:
        
        # pull bio
        bio = searchy(name)
        
        # if no bio, tag as None
        if bio == None:
            
        
        
        
        # and if name isnt in groups table ...
        if members == None:
            
            # check for group status
            
    
    
    
    
    trax.append(rc_jd['tracks']['items'][n]['track']['name'])
    
    q = (a,)
    c.execute('SELECT members FROM groups WHERE name=?', q)
    if c.fetchone() != None:
        a = c.fetchone()[0].split()
    else:
        a = [a]
        
    for m in a:
        if a not in ats:
            ats[a] = {'count':1}
        else:
            ats[a]['count'] += 1
        
    

        
for t in trax:
    if 'feat.' in t:
        feat = re.findall('(?<=feat. ).*[^)]', t)
    elif 'ft.' in t:
        feat = re.findall('(?<=ft. ).*[^)]', t)
    else:
        continue
    for n in re.split('[,&]', feat[0]):
        name = n.strip()
        if name not in ats:
            ats[name] = {'count':1}
        else:
            ats[name]['count'] += 1
    
group_members = False
for name in ats.keys():
#     print(name)
    
    # pull bio
    bio = searchy(name)
    
    # if no bio, tag as None
    if bio == None:
        ats[name]['gender'] = None
    
    # else run pronoun test
    else:
        g_zult = pnoun_test(bio)
#         print(g_zult)
    
        # if tied, pull members to re-run
        if g_zult[0] == g_zult[1]:
            members = searchy(name, group=True)
            if not group_members:
                group_members = []
            for m in members:
                group_members.append(unidecode.unidecode(m))
            ats[name]['gender'] = None
            continue
         
        # else, score and tag
        else:
            if g_zult[0] > g_zult[1]:
                ats[name]['gender'] = 'M'
            elif g_zult[1] > g_zult[0]:
                ats[name]['gender'] = 'F'
                
# if group_members, re-run
if group_members:
    for name in group_members:
        
        # if already in list, just advance counter
        if name in ats:
            ats[name]['count'] += 1
            
        # else, run the whole damn gendering script
        else:
            # pull bio
            bio = searchy(name)

            # if no bio, tag as None
            if bio == None:
                ats[name]['gender'] = None
                
            # else run pronoun test
            # i guess ill have to figure out if groups end up here at some point but wtv
            else:
                g_zult = pnoun_test(bio)
                if g_zult[0] > g_zult[1]:
                    ats[name] = {'count':1, 'gender':'M'}
                elif g_zult[1] > g_zult[0]:
                    ats[name] = {'count':1, 'gender':'F'}
    
# calculate stats
total = 0
M_tot = 0
F_tot = 0
for k, v in ats.items():
    c = v['count']
    total += c
    if v['gender'] == 'M':
        M_tot += c
    elif v['gender'] == 'F':
        F_tot += c
    
print(f'male credits: {M_tot}')
print(f'female credits: {F_tot}')
print(f'total credits: {total}')

In [45]:
# SQL version TAKE 1
ats = {}
trax = []

conn = sqlite3.connect('rgb_test.db')
c = conn.cursor()

for n in range(50):
    a = rc_jd['tracks']['items'][n]['track']['artists'][0]['name']
    trax.append(rc_jd['tracks']['items'][n]['track']['name'])
    
    q = (a,)
    c.execute('SELECT members FROM groups WHERE name=?', q)
    if c.fetchone() != None:
        a = c.fetchone()[0].split()
    else:
        a = [a]
        
    for m in a:
        if a not in ats:
            ats[a] = {'count':1}
        else:
            ats[a]['count'] += 1
        
    

        
for t in trax:
    if 'feat.' in t:
        feat = re.findall('(?<=feat. ).*[^)]', t)
    elif 'ft.' in t:
        feat = re.findall('(?<=ft. ).*[^)]', t)
    else:
        continue
    for n in re.split('[,&]', feat[0]):
        name = n.strip()
        if name not in ats:
            ats[name] = {'count':1}
        else:
            ats[name]['count'] += 1
    
group_members = False
for name in ats.keys():
#     print(name)
    
    # pull bio
    bio = searchy(name)
    
    # if no bio, tag as None
    if bio == None:
        ats[name]['gender'] = None
    
    # else run pronoun test
    else:
        g_zult = pnoun_test(bio)
#         print(g_zult)
    
        # if tied, pull members to re-run
        if g_zult[0] == g_zult[1]:
            members = searchy(name, group=True)
            if not group_members:
                group_members = []
            for m in members:
                group_members.append(unidecode.unidecode(m))
            ats[name]['gender'] = None
            continue
         
        # else, score and tag
        else:
            if g_zult[0] > g_zult[1]:
                ats[name]['gender'] = 'M'
            elif g_zult[1] > g_zult[0]:
                ats[name]['gender'] = 'F'
                
# if group_members, re-run
if group_members:
    for name in group_members:
        
        # if already in list, just advance counter
        if name in ats:
            ats[name]['count'] += 1
            
        # else, run the whole damn gendering script
        else:
            # pull bio
            bio = searchy(name)

            # if no bio, tag as None
            if bio == None:
                ats[name]['gender'] = None
                
            # else run pronoun test
            # i guess ill have to figure out if groups end up here at some point but wtv
            else:
                g_zult = pnoun_test(bio)
                if g_zult[0] > g_zult[1]:
                    ats[name] = {'count':1, 'gender':'M'}
                elif g_zult[1] > g_zult[0]:
                    ats[name] = {'count':1, 'gender':'F'}
    
# calculate stats
total = 0
M_tot = 0
F_tot = 0
for k, v in ats.items():
    c = v['count']
    total += c
    if v['gender'] == 'M':
        M_tot += c
    elif v['gender'] == 'F':
        F_tot += c
    
print(f'male credits: {M_tot}')
print(f'female credits: {F_tot}')
print(f'total credits: {total}')

OperationalError: no such table: groups

In [9]:
!pip install configparser

In [24]:
import sqlite3
import MySQLdb

host = 'rcg.skinnyfriedman.com'
db_user = 'rcg_test'
db_password = 'bMbLMLJ6'
db_name = 'rcg_test'

In [26]:
connection = MySQLdb.connect(host=host, user=db_user, passwd=db_password, db=db_name, port=8886)

OperationalError: (2003, "Can't connect to MySQL server on 'rcg.skinnyfriedman.com' (60)")

In [30]:
connection = sqlite3.connect('rgb_test.db')

In [323]:
import tweepy

In [327]:
consumer_key = 'u8jtUDOxwueEVGLifTnRODip6'
consumer_secret = 'fVAnG7HsU8ueQEjgkzByjqujkr4o1QxRHMs3sVbzysMmB4Xlvw'
access_token = '16603524-RGfQhQHSxqdRFsADqVXopbg0M6aS7a5U0mOP37UVt'
access_token_secret = 'feYeKZc9Vb5FcIEC2DZsBBOnfqnxVIYbrO7roldpLZEsq'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

RT @SeanMcElwee: This bullshit is an incredibly expensive PR campaign orchestrated by billionaires to solidify a far right court that will…
RT @1khustles: HOTBOYBAR - ZIPPER FT. LIL BEAU (OFFICIL VIDEO) https://t.co/ayBETcD9vp via @YouTube
RT @AkiyoshiKitaoka: A demo of lightness perception https://t.co/BSVpgcuIw1
The Inland Printer, February 1895. Cover by Will Bradley. https://t.co/TmmM6c0wT2
RT @1khustles: Zipper out now @hotboybar @_LORDGURU 🎰💰🌍🛰🛫👽 https://t.co/IlbTUcRIou
This completely normal officer raped 45 women and killed 12 of them

https://t.co/LtgguKZIN4
US pure album sales: @thundercat, Drunk 34,000 (total).
RT @KyleNaughtonJr: Issa &amp; Daniel had the most teeth per capita in relationship history.
The Inland Printer, April 1916. https://t.co/2gTKVuUoVX
Our parents have not failed us, our politicians have. 

To any parent reading this: save us, vote, work on politica… https://t.co/cuV3e14eWf
RT @wphillips49: Today we all --journalists and everyday citizens of goodwill-- 

In [328]:
user = api.get_user('skinny412')

In [330]:
print(user.screen_name)
print(user.followers_count)
for friend in user.friends():
   print(friend.screen_name)

skinny412
4061
steeltippeddove
Callietrees
PicturesFoIder
TransitCenter
kennybeats
70sAdultTitles
djtara
DJGigante812
nataliemorales
1followernodad
LateNightSeth
NoFauxCyrillic
RegChapmanJr
TheoLandsman
wybmf
nocontxtmulaney
J__Velasquez
itsmejmg
ami_vet
00ara00


In [353]:
# Let's say this is a web app, so we need to re-build the auth handler
# first...
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
token = session.get('request_token')
session.delete('request_token')
auth.request_token = token

try:
    auth.get_access_token(verifier)
except tweepy.TweepError:
    print('Error! Failed to get access token.')

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.

In [343]:
print(redirect_url)

https://api.twitter.com/oauth/authorize?oauth_token=D-4z9gAAAAAA6PEIAAABZS8XqgE


In [344]:
api = tweepy.API(auth)

In [345]:
api

In [350]:
api.update_status('tweepy + oauth!')

TweepError: Twitter error response: status code = 403